In [ ]:
# descargar librerías sqlalchemy, sqlclient
import warnings
warnings.filterwarnings('ignore')
from sqlalchemy import create_engine
import pandas as pd
import pymysql
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
# Obtener las variables de conexión desde el entorno
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
db = os.getenv("DB_NAME")
excel_path = os.getenv("EXCEL_PATH")

In [ ]:
# Crear la conexión a la base de datos MySQL, antes de hacerlo es necesario crear la base de datos
# y el usuario con los permisos necesarios. Aquí se asume que ya existe una base de datos llamada 'ventasdb'
# y un usuario 'root' con contraseña '12345'
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}", echo=False)
conn = engine.connect()

# Leer todas las hojas del archivo Excel como un diccionario de DataFrames
all_sheets = pd.read_excel(excel_path, sheet_name=None)

In [3]:
print(all_sheets.keys())
hechos_ventas = all_sheets['hechos_ventas'].copy()

dict_keys(['dim_cliente', 'dim_producto', 'dim_region', 'dim_tiempo', 'hechos_ventas'])


In [4]:
# Subir cada hoja como una tabla independiente
for sheet_name, df in all_sheets.items():
    # Eliminar espacios y caracteres especiales del nombre de la hoja para usar como nombre de tabla
    clean_name = sheet_name.strip().lower().replace(" ", "_")
    df.to_sql(clean_name, engine, index=False, if_exists='replace')

# Cerrar la conexión
conn.close()